# Cкачивание текстов для проекта Крюков-Шолохов

## Prepare service data

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os

req_headers={'accept': 'text/plain, text/html',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'cache-control': 'max-age=0',
'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/69.0.3497.81 Chrome/69.0.3497.81 Safari/537.36'}

def text_clear(strText):
    t=re.sub('\n+', ' ', strText)
    t=re.sub('-{3,}', '', t)
    t=re.sub('--', '-', t)
    t=re.sub(' +', ' ', t)
    t=re.sub('<BR>', '\n', t)
    t=re.sub('\n +', '\n', t)
    return t.strip()

In [3]:
%run main_serv.py
auth_load=authors.copy()

In [4]:
auth_load.use(val=False)
auth_load.use(index=['sera', 'fad', 'bab', 'kat', 'pla']) # load these authors

not found param key_word "index" so set value to ALL authors


In [5]:
auth_load

,file_mask,fire,in_use,strFileDir,strRuName,strUrl
index,,,,,,
kru,*,0,False,Krukov,Ф. Крюков,http://az.lib.ru/k/krjukow_f_d/
sho,*.txt,0,False,Sholokhov,М. Шолохов,http://www.lib.ru/PROZA/SHOLOHOW/
ltol,*,0,False,Tolstoy,Л. Толстой,
sera,*,0,True,Serafimovich,А. Серафимович,http://lib.ru/RUSSLIT/SERAFIMOWICH/
fad,*,0,True,Fadeyev,А. Фадеев,http://www.lib.ru/RUSSLIT/FADEEW/
bab,*,0,True,Babel,И. Бабель,http://www.lib.ru/PROZA/BABEL/
kat,*,0,True,Kataev,В. Катаев,http://www.lib.ru/PROZA/KATAEW/
pla,*,0,True,Platonov,А. Платонов,http://platonov-ap.ru/novels/


### Serafimovich

In [6]:
strSERABase=auth_load.loc['sera', 'strUrl']

def read_sera(strSERAURL=strSERABase):
    def work_with_item(strURL, name, sess):
        ser_t=sess.get(strSERABase+strURL)
        ser_t.encoding=ser_t.apparent_encoding
        sp_t=BeautifulSoup(ser_t.text, 'html.parser')
    
        mo=re.search(r'(\n?-{4,} ?\n)', sp_t.text)
        text=sp_t.text[mo.start(1)+mo.end(1):]
    
        text=re.sub('\n  ', '<BR>', text)
        text=re.split('\n?\d+\n', text)
    
        text=list(map(text_clear, text))
    
        for n, t in enumerate(text[:-1]):
            strFileName=os.path.join(auth_load.loc['sera', 'strFileDir'], '{name}_{num}.txt'.format(name=name, num=n+1))
            print('work with ', strFileName, end=' ')
        
            with open(strFileName, 'w') as flw:
                flw.write(t)
                print('Done', end='\n')
        return text

    with requests.session() as sess:
        sess.headers.update(req_headers)
        ser_m=sess.get(strSERAURL)
        ser_m.encoding=ser_m.apparent_encoding
        sp_ser=BeautifulSoup(ser_m.text, 'html.parser')

        links=sp_ser.findAll(lambda x: x.name == 'a' and re.search(r'\.txt\b', x['href']))

        try:
            os.mkdir(auth_load.loc['sera', 'strFileDir'])
        except:
            pass
    
        for link in links:
            work_with_item(link['href'], link.text, sess)

        print('All done.')
        
#read_sera()

### Fadeyev

In [7]:
strFADURL=auth_load.loc['fad', 'strUrl']

with requests.session() as sess:
    sess.headers.update(req_headers)
    fad_m=sess.get(strFADURL)
    fad_m.encoding=fad_m.apparent_encoding
    sp_fad=BeautifulSoup(fad_m.text, 'html.parser')

    #links=sp_fad.findAll(lambda x: x.name == 'a' and re.search(r'\.txt\b', x.get('href')))
    links=[l for l in sp_fad.findAll('a', href=True) if re.search(r'\.txt\b', l['href'])]

    try:
        os.mkdir(auth_load.loc['fad', 'strFileDir'])
    except:
        pass
    
    for link in links:
        print(link['href'], link.text)

    print('All done.')

razgrom.txt Разгром
mol_gwardiya.txt Молодая гвардия
amgun.txt Рождение амгуньского полка
bednbogt.txt О бедности и богатстве
metal.txt Черная металлургия
odin.txt Один в чаще
razliv.txt Разлив
udege.txt Последний из удэге
zemletr.txt Землетрясение
ofadeeve.txt В.Озеров. Певец революционной эпохи
All done.


In [8]:
s=sess.get(strFADURL+links[0]['href'])

s.encoding=s.apparent_encoding
sp=BeautifulSoup(s.text, 'html.parser')

text_html=re.split('<ul><a name=\d+></a><h2>[IVXА-Яа-я., ]+</h2></ul>', s.text)
text=[t.strip() for t in text_html if not (re.search(r'(\n?-{4,} ?\n)', t) or re.search(r'<form action=', t))]

for i in text:
    print(i[:250])

Бренча по ступенькам избитой японской шашкой, Левинсон вышел во двор. С
полей тянуло гречишным медом. В жаркой бело-розовой пене плавало над головой
июльское солнце.
     Ординарец  Морозка,  отгоняя  плетью  осатаневших   цесарок,  сушил  на
брезент
Сказать правду, спасенный не понравился Морозке с первого взгляда.
     Морозка не  любил чистеньких людей.  В его  жизненной практике это были
непостоянные, никчемные люди,  которым нельзя верить. Кроме того,  раненый с
первых же шагов проявил себя 
Морозка и Варя вернулись за полдень, не глядя друг на  друга, усталые и
ленивые.
     Морозка вышел на  прогалину и, заложив  два пальца в  рот, свистнул три
раза пронзительным разбойным свистом. И когда, как в сказке, вылетел из чащи
курчавый, звонк
Приезд Морозки нарушил душевное равновесие, установившееся в Мечике под
влиянием ровной, безмятежной жизни в госпитале.
     "Почему  он  смотрел  так  пренебрежительно? -- подумал  Ме-чик,  когда
ординарец уехал. --  Пусть он вытащил меня из  огн

In [218]:
def work_with_item(strURL, name, sess):

    s=sess.get(strURL)

    s.encoding=s.apparent_encoding
    sp=BeautifulSoup(s.text, 'html.parser')

    text_html=re.sub('\n  ', '<BR>', s.text)
    
    text_html=re.split('<ul><a name=\d+></a><h2>[0-9IVXА-Яа-я?!*., ]+</h2></ul>', text_html)
    
    text=[t.strip() for t in text_html if not (re.search(r'(\n?-{4,} ?\n)', t) or re.search(r'<form action=', t))]

    text=list(map(text_clear, text))
    
    for n, t in enumerate(text):
        strFileName=os.path.join(auth_load.loc['fad', 'strFileDir'], '{name}_{num}.txt'.format(name=name, num=n+1))
        print('work with ', strFileName, end=' ')
        
        with open(strFileName, 'w') as flw:
            flw.write(t)
            print('Done', end='\n')


In [228]:
ind=9
work_with_item(strFADURL+links[ind]['href'], links[ind].text, sess)

work with  Fadeyev/В.Озеров. Певец революционной эпохи_1.txt Done
work with  Fadeyev/В.Озеров. Певец революционной эпохи_2.txt Done
work with  Fadeyev/В.Озеров. Певец революционной эпохи_3.txt Done
work with  Fadeyev/В.Озеров. Певец революционной эпохи_4.txt Done


## Шолохов: доработка

In [39]:
strROD=r'Они сражались за Родину.txt'
strROD_t=r'Они сражались за Родину-{0}.txt'
with open(auth_load.loc['sho', 'strFileDir'] +'/' + strROD, 'r') as fl:
    text=fl.read()
    
    text=re.sub('\n  ', '<BR>', text)
    text=re.split('\n<BR>', text)
    text=list(map(text_clear, text))

    for n, t in enumerate(text):
        with open(auth_load.loc['sho', 'strFileDir'] +'/' + strROD_t.format(n), 'w') as fw:
            fw.write(t)

print('all done')

all done


In [43]:
import glob

lst_kru=glob.glob(auth_load.loc['kru', 'strFileDir'] + '/*')
for i in lst_kru:
    print(i)

Krukov/Новым строем
Krukov/Обыватель
Krukov/Ответственность момента
Krukov/Шквал
Krukov/Встреча
Krukov/Итальянец Замчалов
Krukov/Долг перед Родиной
Krukov/Товарищи
Krukov/Забытые слова
Krukov/К изучению родного края
Krukov/Булавинский бунт. (1707-08 гг.)
Krukov/Из дневника учителя Васюхина
Krukov/Душа одна
Krukov/Мать
Krukov/Новочеркасск, 7 апреля
Krukov/Казачка
Krukov/В нынешние светлые лунные ночи...
Krukov/Группа Б
Krukov/Без огня
Krukov/Зов братьев
Krukov/Единое на потребу
Krukov/Тишь
Krukov/Чувство чести и достоинства
Krukov/Полчаса
Krukov/Живые вести
Krukov/Сеть мирская
Krukov/Шульгинская расправа
Krukov/Сила духа
Krukov/Край родной
Krukov/М.П. Богаевский
Krukov/В сфере колдовства и мути
Krukov/Жажда
Krukov/Визитка от Арона Бибера
Krukov/Жертва Каледина
Krukov/Знамя Мануила Семилетова
Krukov/В сугробах
Krukov/Здесь и там
Krukov/Офицерша
Krukov/Свежо предание
Krukov/Четверо
Krukov/У окна
Krukov/29 января 1918 - 29 января 1919
Krukov/Итоги дней 5-7 ноября
Krukov/Жертвы Искупительны